In [1]:
import caffe

import numpy as np
from numpy import array
from numpy.random import normal
from matplotlib import pyplot
import os, sys
import struct

caffe.set_device(0)  
caffe.set_mode_gpu()


model_def = '/home/jerry/caffe-bvlc/examples/bvlc_alexnet/train_val.prototxt'
# model_def = '/home/jerry/caffe-bvlc/examples/bvlc_alexnet/deploy.prototxt'
model_weights = '/home/jerry/caffe-bvlc/examples/bvlc_alexnet/bvlc_alexnet.caffemodel'



In [2]:
f_path = './npy/c5_2d.npy'
c5_2d = np.load(f_path)
print c5_2d.shape
w5_coeff  = c5_2d.reshape(256, 192, 3, 3)

(256, 1728)


In [3]:
def rate(num, p):
    s = 0
    for i in num[15:-p:-1]:
        s += i

    return (s+num[0])/num.sum(-1)

In [4]:
## Only coefff
p_coeff = -7

param = [0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5,1.0e-6, 1.0e-7, 1.0e-8, 1.0e-9, 1.0e-10, 1.0e-11, 1.0e-12, 1.0e-13, 1.0e-14, 1.0e-15]
# param = 0
idx = w5_coeff < param[-p_coeff]

mask_in_kernel = np.ones(256* 192*3*3, dtype=np.float32).reshape(256, 192, 3, 3)
mask_in_kernel[idx] = 0
print mask_in_kernel[0].shape  ##(192, 3, 3)

## Prune In Kernel
net_in_kernel = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

w5_in_kernel = net_in_kernel.params['conv5'][0].data
b5_in_kernel = net_in_kernel.params['conv5'][1].data
print w5_in_kernel.shape
print b5_in_kernel.shape

w5_prune_in_kernel = w5_in_kernel * mask_in_kernel
net_in_kernel.params['conv5'][0].data[...] = w5_prune_in_kernel
# prune_in_kernel_model = '/home/jerry/caffe-bvlc/examples/bvlc_alexnet/alexnet_pc50_in_kernel.caffemodel'
# net_in_kernel.save(prune_in_kernel_model)

(192, 3, 3)
(256, 192, 3, 3)
(256,)


In [5]:
# p_rate = 0.3107
p_rate = 0.5
p_num = int(256*1728*p_rate)
print p_num

221184


In [6]:
## Ci *Wi

net_c5w5 = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

w5_c5w5 = net_c5w5.params['conv5'][0].data
# b5_c5w5 = net_c5w5.params['conv5'][1].data
c5_w5 =w5_coeff*w5_c5w5
c5_w5_1d =abs(c5_w5).reshape(256*1728)
c5_w5_1d.sort(axis=0)

idx_c5w5 = abs(c5_w5) < c5_w5_1d[p_num - 1]

mask_c5w5 = np.ones(256* 192*3*3, dtype=np.float32).reshape(256, 192, 3, 3)
mask_c5w5[idx_c5w5] = 0
# print mask_c5w5[0].shape  ##(192, 3, 3)

net_c5w5.params['conv5'][0].data[...] = w5_c5w5 * mask_c5w5
# print w5_prune_c5w5[0]
# print '###'
# print w5_c5w5[0]

In [7]:
## Song Han



net_Song = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST) 
w5_Song = net_Song.params['conv5'][0].data

w5_1d = w5_Song.reshape(256*1728)
w5_1d = abs(w5_1d)
w5_1d.sort(axis=0)  ## for small to big
# print w5_1d[0]

# print w5_Song.sort

print w5_1d.shape
print w5_1d


p_param = w5_1d[p_num - 1]
print p_param
idx_Song = abs(w5_Song) < p_param
mask_Song = np.ones(256* 192*3*3, dtype=np.float32).reshape(256, 192, 3, 3)
mask_Song[idx_Song] = 0

net_Song.params['conv5'][0].data[...] =w5_Song * mask_Song


(442368,)
[1.4089139e-08 1.7953059e-07 3.3432823e-07 ... 3.0816948e-01 3.2797512e-01
 3.3826050e-01]
0.011744433


In [8]:
print np.count_nonzero(mask_c5w5)
print np.count_nonzero(mask_Song)

221185
221185


In [24]:
net_base = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

# p_net = caffe.Net(model_def,      # defines the structure of the model
#                 prune_kernel_model,  # contains the trained weights
#                 caffe.TEST)     # use test mode (e.g., don't perform dropout)

# pc50_net = caffe.Net(model_def,      # defines the structure of the model
#                 prune_in_kernel_model,  # contains the trained weights
#                 caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [23]:
def loss_acc_test(net, test_iter):
    loss = np.zeros(1)
    acc = np.zeros(1)
    test_loss = 0
    test_accuracy = 0
    for test_it in range(test_iter):
        net.forward()

        test_loss += net.blobs['loss'].data    
        test_accuracy += net.blobs['accuracy'].data    
        loss[0] = test_loss / test_iter 
        acc[0] = test_accuracy / test_iter
#     return loss[0], acc[0]  
    print 'Loss: ', loss[0], 'Accuracy:', acc[0]

iter_ = 100

print 'Benchmark:'
loss_acc_test(net_base, iter_)

loss_acc_test(net_c5w5, iter_)
loss_acc_test(net_Song, iter_)

Loss:  0.9318228471279144 Accuracy: 0.7793999963998794
Loss:  1.7995616042613982 Accuracy: 0.6259999999403953
Loss:  0.9734689846634865 Accuracy: 0.7597999966144562


In [33]:
print 'Benchmark:', 'Loss', test_loss, 'Accuracy', test_acc
# print 'Only aa Coeff:', 'Loss', pc50_test_loss, 'Accuracy', pc50_test_acc
print 'Coeff*Weights:', 'Loss',  c5w5_test_loss, 'Accuracy', c5w5_test_acc, 'Pruning Rate: 50%'
print 'Song:', 'Loss',  song_test_loss, 'Accuracy', song_test_acc, 'Pruning Rate: 50%'

Benchmark: Loss [0.92719088] Accuracy [0.77584]
Coeff*Weights: Loss [1.84362456] Accuracy [0.61764] Pruning Rate: 50%
Song: Loss [0.96775612] Accuracy [0.76844] Pruning Rate: 50%


In [ ]:
if (c5w5_test_acc[0] > song_test_acc[0]): print 'c5w5 is bigger'

In [ ]:
def prune_caffe_test():

In [30]:



test_iter = 1000

test_loss = np.zeros(1)
test_acc = np.zeros(1)
_test_loss = 0
_accuracy = 0

# p_test_loss = np.zeros(1)
# p_test_acc = np.zeros(1)
# _p_test_loss = 0
# _p_accuracy = 0

pc50_test_loss = np.zeros(1)
pc50_test_acc = np.zeros(1)
_pc50_test_loss = 0
_pc50_accuracy = 0

c5w5_test_loss = np.zeros(1)
c5w5_test_acc = np.zeros(1)
_c5w5_test_loss = 0
_c5w5_accuracy = 0


song_test_loss = np.zeros(1)
song_test_acc = np.zeros(1)
_song_test_loss = 0
_song_accuracy = 0

# p_test_loss = np.zeros(1)
# p_test_acc = np.zeros(1)
# _p_test_loss = 0
# _p_accuracy = 0



for test_it in range(test_iter):  

    net_base.forward() 
    _test_loss += net_base.blobs['loss'].data    
    _accuracy += net_base.blobs['accuracy'].data    
    test_loss[0] = _test_loss / test_iter 
    test_acc[0] = _accuracy / test_iter   
    
    
#     net_kernel.forward()
#     _p_test_loss += net_kernel.blobs['loss'].data    
#     _p_accuracy += net_kernel.blobs['accuracy'].data    
#       p_test_acc[0] = _p_accuracy / test_iter  
#     p_test_loss[0] = _p_test_loss / test_iter
    
#     net_in_kernel.forward()  
#     # 计算test loss  
#     _pc50_test_loss += net_in_kernel.blobs['loss'].data   
#     # 计算test accuracy  
#     _pc50_accuracy += net_in_kernel.blobs['accuracy'].data
#     # 计算平均test loss  
#     pc50_test_loss[0] = _pc50_test_loss / test_iter
#     # 计算平均test accuracy  
#     pc50_test_acc[0] = _pc50_accuracy / test_iter
    
    net_c5w5.forward()
    _c5w5_test_loss += net_c5w5.blobs['loss'].data    
    _c5w5_accuracy += net_c5w5.blobs['accuracy'].data    
    c5w5_test_loss[0] = _c5w5_test_loss / test_iter 
    c5w5_test_acc[0] = _c5w5_accuracy / test_iter 
    
    net_Song.forward()
    _song_test_loss += net_Song.blobs['loss'].data    
    _song_accuracy += net_Song.blobs['accuracy'].data    
    song_test_loss[0] = _song_test_loss / test_iter 
    song_test_acc[0] = _song_accuracy / test_iter 
    
    
print 'Benchmark:', 'Loss', test_loss[0], 'Accuracy', test_acc[0]
# print 'Only aa Coeff:', 'Loss', pc50_test_loss[0], 'Accuracy', pc50_test_acc[0]
print 'Coeff*Weights:', 'Loss',  c5w5_test_loss[0], 'Accuracy', c5w5_test_acc[0]
print 'Song:', 'Loss',  song_test_loss[0], 'Accuracy', song_test_acc[0]

Benchmark: Loss 0.9271908807456494 Accuracy 0.7758399975299836
Coeff*Weights: Loss 1.8436245554685593 Accuracy 0.6176400049328804
Song: Loss 0.9677561219632625 Accuracy 0.768439998626709


## Output Log:

### w5_coeff statistics
```
1.0e-4:  0.0
1.0e-5:  0.0
1.0e-6:  2507.0
1.0e-7:  347714.0
1.0e-8:  84944.0
1.0e-9:  4360.0
1.0e-10:  394.0
1.0e-11:  37.0
1.0e-12:  2.0
1.0e-13:  1.0
1.0e-14:  0.0
1.0e-15:  0.0
0:  2409.0

Total: 442368.0
```
###  1. w5_coeff < 1.0e-9

#### iter = 1000
Benchmark: 
Loss 0.9271908807456494 Accuracy 0.7758399975299836

Lab Prune:
Loss 0.9321347722709179 Accuracy 0.7750199978351593

###  2. w5_coeff < 1.0e-7

#### iter = 1000

Benchmark: 
Loss 0.9271908807456494 Accuracy 0.7758399975299836

Lab Prune: 
Loss 1.6888461483716966 Accuracy 0.6386000045239926

###  3. w5_coeff < 1.0e-6
Benchmark: 
Loss 0.9271908807456494 Accuracy 0.7758399975299836

Lab Prune: 
Loss 6.850017457485199 Accuracy 0.007959999822080135

In [28]:
b_mean_all_1d = c5_2d.reshape(256*1728)
num = np.zeros((16), dtype = np.float32)

for j in range(256*1728):
    if b_mean_all_1d[j] == 0:
        num[0] += 1  
    elif 0< abs(b_mean_all_1d[j]) < 1.0e-14:
        num[15] += 1
    elif 1.0e-14< abs(b_mean_all_1d[j]) < 1.0e-13: 
        num[14] += 1
    elif 1.0e-13< abs(b_mean_all_1d[j]) < 1.0e-12:
        num[13] += 1
    elif 1.0e-12< abs(b_mean_all_1d[j]) < 1.0e-11:
        num[12] += 1
    elif 1.0e-11< abs(b_mean_all_1d[j]) < 1.0e-10:   
        num[11] += 1
    elif 1.0e-10< abs(b_mean_all_1d[j]) < 1.0e-9:   
        num[10] += 1
    elif 1.0e-9 <= abs(b_mean_all_1d[j]) < 1.0e-8:
        num[9] += 1
    elif 1.0e-8 <= abs(b_mean_all_1d[j])  < 1.0e-7:
        num[8] += 1
    elif 1.0e-7 <= abs(b_mean_all_1d[j])  < 1.0e-6:
        num[7] += 1
    elif 1.0e-6 <= abs(b_mean_all_1d[j]) < 1.0e-5:
        num[6] += 1
    elif 1.0e-5 <= abs(b_mean_all_1d[j])  < 1.0e-4:
        num[5] += 1
    elif 1.0e-4 <= abs(b_mean_all_1d[j])  < 1.0e-3:
        num[4] += 1    
    else :
        n_others += 1
            


print '1.0e-4: ',  num[4]
print '1.0e-5: ',  num[5]
print '1.0e-6: ',  num[6]
print '1.0e-7: ',  num[7]
print '1.0e-8: ',  num[8]
print '1.0e-9: ',  num[9]
print '1.0e-10: ',  num[10]
print '1.0e-11: ',  num[11]
print '1.0e-12: ',  num[12]
print '1.0e-13: ',  num[13]
print '1.0e-14: ',  num[14]
print '1.0e-15: ',  num[15]
print '0: ',  num[0]
print #######
print 'Total:', np.sum(num, -1)
if (np.sum(num, -1) == 256*1728): print 'Total No. Correct!'

1.0e-4:  0.0
1.0e-5:  0.0
1.0e-6:  2507.0
1.0e-7:  347714.0
1.0e-8:  84944.0
1.0e-9:  4360.0
1.0e-10:  394.0
1.0e-11:  37.0
1.0e-12:  2.0
1.0e-13:  1.0
1.0e-14:  0.0
1.0e-15:  0.0
0:  2409.0

Total: 442368.0
Total No. Correct!


In [ ]:
c5_w5_1d = c5_w5.reshape(256*1728)
num_c5w5 = np.zeros((16), dtype = np.float32)

for j in range(256*1728):
    if c5_w5_1d[j] == 0:
        num_c5w5[0] += 1  
    elif 0< abs(c5_w5_1d[j]) < 1.0e-14:
        num_c5w5[15] += 1
    elif 1.0e-14< abs(c5_w5_1d[j]) < 1.0e-13: 
        num_c5w5[14] += 1
    elif 1.0e-13< abs(c5_w5_1d[j]) < 1.0e-12:
        num_c5w5[13] += 1
    elif 1.0e-12< abs(c5_w5_1d[j]) < 1.0e-11:
        num_c5w5[12] += 1
    elif 1.0e-11< abs(c5_w5_1d[j]) < 1.0e-10:   
        num_c5w5[11] += 1
    elif 1.0e-10< abs(c5_w5_1d[j]) < 1.0e-9:   
        num_c5w5[10] += 1
    elif 1.0e-9 <= abs(c5_w5_1d[j]) < 1.0e-8:
        num_c5w5[9] += 1
    elif 1.0e-8 <= abs(c5_w5_1d[j])  < 1.0e-7:
        num_c5w5[8] += 1
    elif 1.0e-7 <= abs(c5_w5_1d[j])  < 1.0e-6:
        num_c5w5[7] += 1
    elif 1.0e-6 <= abs(c5_w5_1d[j]) < 1.0e-5:
        num_c5w5[6] += 1
    elif 1.0e-5 <= abs(c5_w5_1d[j])  < 1.0e-4:
        num_c5w5[5] += 1
    elif 1.0e-4 <= abs(c5_w5_1d[j])  < 1.0e-3:
        num_c5w5[4] += 1    
    else :
        n_others += 1
            

print '系数与权重乘积：'
print '1.0e-4: ',  num_c5w5[4]
print '1.0e-5: ',  num_c5w5[5]
print '1.0e-6: ',  num_c5w5[6]
print '1.0e-7: ',  num_c5w5[7]
print '1.0e-8: ',  num_c5w5[8]
print '1.0e-9: ',  num_c5w5[9]
print '1.0e-10: ',  num_c5w5[10]
print '1.0e-11: ',  num_c5w5[11]
print '1.0e-12: ',  num_c5w5[12]
print '1.0e-13: ',  num_c5w5[13]
print '1.0e-14: ',  num_c5w5[14]
print '1.0e-15: ',  num_c5w5[15]
print '0: ',  num_c5w5[0]
print #######
print 'Total:', np.sum(num_c5w5, -1)
if (np.sum(num_c5w5, -1) == 256*1728): print 'Total No. Correct!'


In [ ]:
print rate(num, p_coeff)
print rate(num_c5w5, p_w5c5)

In [ ]:
p = -12 ## <1.0e-12 num[13]
num_index = -p +1
print num_index
param =np.array([0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5,1.0e-6, 1.0e-7, 1.0e-8, 1.0e-9, 1.0e-10, 1.0e-11, 1.0e-12, 1.0e-13, 1.0e-14, 1.0e-15])

s = 0
for i in num[15:-p:-1]:
    s += i
    print i
print s

In [ ]:
a = np.array([1,2,3,4,5,6,7,8])
a.sum(0,range(0,2))

Benchmark:  
Loss [0.94938668] Accuracy [0.762] Prune rate: 0

只考虑aa系数的prune: 
Loss [1.70731913] Accuracy [0.67]  Prune rate: 20.83%

同时考虑aa系数和权重值(二者的乘积): 
Loss [1.1640587] Accuracy [0.732] Prune rate: 31.07%


In [ ]:
系数与权重乘积：
1.0e-4:  0.0
1.0e-5:  0.0
1.0e-6:  0.0
1.0e-7:  321.0
1.0e-8:  40066.0
    
--------------------- rate 90.87% Accuracy 0.0079
1.0e-9:  264537.0
--------------------- rate 31.07% Accuracy 0.732
1.0e-10:  115511.0
1.0e-11:  17060.0
1.0e-12:  2146.0
1.0e-13:  288.0
1.0e-14:  25.0
1.0e-15:  5.0
0:  2409.0
    


In [26]:
a = np.ones(256*1728)
len(a)

442368

In [27]:
def range_statis(array_1d):
    
    num = np.zeros((16), dtype = np.float32)
    for j in range(len(array_1d)):
        if array_1d[j] == 0:
            num[0] += 1  
        elif 0< abs(array_1d[j]) < 1.0e-14:
            num[15] += 1
        elif 1.0e-14< abs(array_1d[j]) < 1.0e-13: 
            num[14] += 1
        elif 1.0e-13< abs(array_1d[j]) < 1.0e-12:
            num[13] += 1
        elif 1.0e-12< abs(array_1d[j]) < 1.0e-11:
            num[12] += 1
        elif 1.0e-11< abs(array_1d[j]) < 1.0e-10:   
            num[11] += 1
        elif 1.0e-10< abs(array_1d[j]) < 1.0e-9:   
            num[10] += 1
        elif 1.0e-9 <= abs(array_1d[j]) < 1.0e-8:
            num[9] += 1
        elif 1.0e-8 <= abs(array_1d[j])  < 1.0e-7:
            num[8] += 1
        elif 1.0e-7 <= abs(array_1d[j])  < 1.0e-6:
            num[7] += 1
        elif 1.0e-6 <= abs(array_1d[j]) < 1.0e-5:
            num[6] += 1
        elif 1.0e-5 <= abs(array_1d[j])  < 1.0e-4:
            num[5] += 1
        elif 1.0e-4 <= abs(array_1d[j])  < 1.0e-3:
            num[4] += 1    
        else :
            pass



    print '1.0e-4: ',  num[4]
    print '1.0e-5: ',  num[5]
    print '1.0e-6: ',  num[6]
    print '1.0e-7: ',  num[7]
    print '1.0e-8: ',  num[8]
    print '1.0e-9: ',  num[9]
    print '1.0e-10: ',  num[10]
    print '1.0e-11: ',  num[11]
    print '1.0e-12: ',  num[12]
    print '1.0e-13: ',  num[13]
    print '1.0e-14: ',  num[14]
    print '1.0e-15: ',  num[15]
    print '0: ',  num[0]
    print #######
    print 'Total:', np.sum(num, -1)
#     if (np.sum(num, -1) == 256*1728): print 'Total No. Correct!'

In [29]:
range_statis(b_mean_all_1d)

1.0e-4:  0.0
1.0e-5:  0.0
1.0e-6:  2507.0
1.0e-7:  347714.0
1.0e-8:  84944.0
1.0e-9:  4360.0
1.0e-10:  394.0
1.0e-11:  37.0
1.0e-12:  2.0
1.0e-13:  1.0
1.0e-14:  0.0
1.0e-15:  0.0
0:  2409.0

Total: 442368.0
